In [17]:
import pygame
import numpy as np
import matplotlib.pyplot as plt
import random

pygame.init()

(6, 0)

In [18]:
def generate_danger_locations(number_of_locations):
    Danger_location = []
    for i in range(number_of_locations):
        location =(random.randint(20,1180),random.randint(20,580))
        Danger_location.append(location)
    return Danger_location

In [24]:
Screen_width = 1200
Screen_height = 600
Start_position = (600,300)
Goal_position = (500,300)
number_of_danger_locations = 10
Danger_location = generate_danger_locations(number_of_danger_locations)
Actions = {0:"left",1:"right",2:"up",3:"down"}
game_over = False
exit_game = False
white = (255,255,255)
red = (255,0,0)
black = (0,0,0)
blue = (125,125,125)
agent_size = 20
danger_size = 20
fps = 30
jump_size = 5

In [25]:
class Enviornment:
    def __init__(self):
        self.danger_locations =Danger_location
        self.start_pos =Start_position
        self.goal_pos = Goal_position
        self.actions =Actions
        self.width = Screen_width
        self.height = Screen_height
        self.state_action_values = {}
        self.current_position = Start_position
        self.reward = 0
        for x in range(self.width):
            for y in range(self.height):
                state = (x,y)
                self.state_action_values[state] = [0.25 for i in range(len(self.actions))]
                
        self.state_action_values[self.goal_pos] = [0.0 for i in range(len(self.actions))]
        
    def nextState(self,action,current_state):
        if self.actions[action] == "left":
            newPossiblePos = (current_state[0]-jump_size,current_state[1])
            if newPossiblePos[0]>0:
                self.current_position = newPossiblePos
            else:
                pass
                
        elif self.actions[action] == "right":
            newPossiblePos = (current_state[0]+jump_size,current_state[1])
            if newPossiblePos[0]<self.width:
                self.current_position = newPossiblePos
            else:
                pass
                
        elif self.actions[action] == "up":
            newPossiblePos = (current_state[0],current_state[1]-jump_size)
            if newPossiblePos[0]>0:
                self.current_position = newPossiblePos
            else:
                pass
                
        elif self.actions[action] == "down":
            newPossiblePos = (current_state[0],current_state[1]+jump_size)
            if newPossiblePos[0]<self.height:
                self.current_position = newPossiblePos
            else:
                pass
                
        return self.current_position
    
    def giveReward(self):
        if abs(self.current_position[0]-self.goal_pos[0])< agent_size or abs(self.current_position[1]-self.goal_pos[1])< agent_size:
            self.reward = 100
        elif self.current_position in self.danger_locations:
            selfreward = -100
        elif self.current_position not in self.danger_locations:
            for state in self.danger_locations:
                if abs(self.current_position[0]-state[0]) < agent_size or abs(self.current_position[1]-state[1])< agent_size:
                    self.reward = -10
        else:
            self.reward = 0
        return self.reward

In [26]:
class Agent:
    def __init__(self,discount_factor = 0.9, alpha = 0.1, epsilon = 0.2, max_steps = 200, num_episodes = 10):
        self.enviornment = Enviornment()
        self.epsilon = epsilon
        self.num_episodes=num_episodes
        self.alpha = alpha
        self.max_steps = max_steps
        self.gamma = discount_factor
        self.TotalReward = 0
        self.Totalreward_per_episode = [] 
        self.Terminal = False
        
    def chooseAction(self,state):
        StateActionValues = self.enviornment.state_action_values
        action_values = StateActionValues[state]
        if self.epsilon < random.random():
            action_values = np.asarray(action_values)
            choosen_action = np.argmax(action_values)
        else:
            choosen_action = random.randint(0,len(self.enviornment.actions)-1)
        return choosen_action
    
    def play_game(self):
        exit_game = False
        game_over = False
        clock = pygame.time.Clock()
        gameWindow = pygame.display.set_mode((Screen_width,Screen_height))
        pygame.display.set_caption("Life_runner")
        pygame.display.update()
        font = pygame.font.SysFont(None,55)
        for i in range(self.num_episodes):
            S = Start_position
            agent_pos_x = Start_position[0]
            agent_pos_y = Start_position[1]
            steps = 0
            self.Terminal = False
            exit_game = False
            while not exit_game and steps < self.max_steps and self.Terminal != True:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        exit_game = True

                gameWindow.fill(white)#fill the window with white color
                pygame.draw.rect(gameWindow,blue,[agent_pos_x,agent_pos_y,agent_size,agent_size])
                pygame.draw.rect(gameWindow,black,[self.enviornment.goal_pos[0],self.enviornment.goal_pos[1],agent_size,agent_size])
                for state in self.enviornment.danger_locations:
                    pygame.draw.rect(gameWindow,red,[state[0],state[1],danger_size,danger_size])
                
                A = self.chooseAction(S)
                S_prime = self.enviornment.nextState(A,S)
                self.enviornment.current_position = S_prime
                R = self.enviornment.giveReward()   
                agent_pos_x,agent_pos_y = S_prime[0],S_prime[1] 
                
                if S_prime == Goal_position:
                    self.Terminal = True
                    self.enviornment.state_action_values[S][A] += self.alpha*(R -self.enviornment.state_action_values[S][A])
                    break
                else:
                    self.enviornment.state_action_values[S][A] += self.alpha*(R +self.gamma*max(self.enviornment.state_action_values[S_prime])-self.enviornment.state_action_values[S][A])
                S = S_prime
                steps += 1
                pygame.display.update()
                clock.tick(fps)
        pygame.quit()
        
    def return_stateActionValues(self):
        return self.enviornment.state_action_values

In [27]:
agent = Agent()